In [1]:
import itertools
import json
import os
import requests

from nltk.tokenize import sent_tokenize
import spacy

In [2]:
spacy_fn = spacy.load("en_core_sci_md", disable=["lemmatizer", "ner"])

In [3]:
ANNOTATION_SERVICE_URL = "http://localhost:9341/predict/semantics/"

In [4]:
def annotate_text(text, max_retries=5):
    response = requests.get(ANNOTATION_SERVICE_URL, params={"utterance": text})
    retries = 0
    while response.status_code != 200 and retries < max_retries:
        response = requests.get(ANNOTATION_SERVICE_URL, params={"utterance": text})
        retries += 1
    if retries >= max_retries:
        print(text)
        return None
    return response.json()
        
    tokens = response.json()["tokens"]
    for token in tokens:
        if token["label"] is None:
            continue
        start_token = token["start"]
        end_token = token["end"]
        for token_idx in range(start_token, end_token + 1):
            token_annotation = Annotation()
            token_annotation.token_no=token_idx
            token_annotation.vn_class=token["label"]
            token_annotation.verb=token["text"]
            return_sentence.add_annotation(token_annotation)
    return return_sentence

In [5]:
data_dir = "/projects/flow_graphs/data"
datasets = ["wsj", "risec", "japflow", "chemu", "mscorpus" ]

In [6]:
{
    "wsj": lambda x: x.split("\n"),
    "risec": lambda x: x.split("\n"),
    "japflow": sent_tokenize,
    "chemu": lambda x: [sent.text for sent in spacy_fn(x).sents]
    "mscorpus": lambda x: [sent.text for sent in spacy_fn(x).sents]
    
}

{'wsj': <function __main__.<lambda>(x)>,
 'risec': <function __main__.<lambda>(x)>,
 'japflow': <function nltk.tokenize.sent_tokenize(text, language='english')>,
 'chemu': <function __main__.<lambda>(x)>}

In [7]:
dataset = "mscorpus"

with open(os.path.join(data_dir, dataset, "train.json")) as f:
    data = json.load(f)

In [8]:
[sent.text for sent in spacy_fn(data[35]["text"]).sents]

['10.1016/j.apcata.2006.05.025\nUnusual CH4 dry reforming performance over Ni-based catalyst separated from nickel metal hydride battery waste\nNi-MH waste used here was collected from the crucible for producing Ni-MH battery provided from Mitsui Mining and Smelting Co. Ltd., Japan.',
 'In our previous study, Ni-based compound was successfully separated from Ni-MH waste by a series of acid and base treatments [Environ. Sci.',
 'Technol., submitted].',
 'Namely, Ni-MH waste of 10 g was dissolved with 2 M (mol/dm3) HCl solution of 200 ml for 24 h. Then, the dissolved transition and rare-earth metal ions were precipitated at pH 12 with 2 M NaOH solution.',
 'The Ni component was extracted from this precipitate by treating with 7.5 M NH3 aq.',
 'of 200 ml, through forming a Ni-ammonium complex.',
 'After filtration, the purple filtrate was heated around 60 degC to decompose the Ni-ammonium complex and to obtain Ni(OH)2.',
 'The sample, s-NiO, was obtained by calcination of the resulting Ni

In [9]:
text = data[35]["text"].split("\n")[0]
print(text)
annotate_text(text)

10.1016/j.apcata.2006.05.025


{'tokens': [{'index': 0,
   'isPredicate': False,
   'isModifier': False,
   'label': None,
   'start': 0,
   'end': 0,
   'text': '10.1016 / j.apcata .2006.05.025 .',
   'pb': None,
   'vn': None,
   'description': None,
   'modifier': False,
   'predicate': False}],
 'props': []}